# Model


In [24]:
import glob
import os
import json
from datetime import datetime
import math
import requests

### Load weather data from JSON


In [16]:
list_of_files = glob.glob("data/*")
latest_file = max(list_of_files, key=os.path.getctime)

with open(latest_file, "r") as f:
    data = f.read()
    data = json.loads(data)

# get clouds data for next hour
next_hour = datetime.now().hour + 1
low_clouds = data["data_1h"]["lowclouds"][next_hour]
mid_clouds = data["data_1h"]["midclouds"][next_hour]
high_clouds = data["data_1h"]["highclouds"][next_hour]

### Obtain today's sunrise/sunset time


In [22]:
LAT = 31.09892
LONG = 121.280426
url = f"https://api.sunrisesunset.io/json?lat={LAT}&lng={LONG}"
response = requests.get(url)
response = response.json()
sunrise = datetime.strptime(
    response["results"]["date"] + " " + response["results"]["sunrise"],
    "%Y-%m-%d %I:%M:%S %p",
)
sunset = datetime.strptime(
    response["results"]["date"] + " " + response["results"]["sunset"],
    "%Y-%m-%d %I:%M:%S %p",
)

Sunrise: 2024-06-07 04:52:10
Sunset: 2024-06-07 18:58:09


### Calculate variables


In [ ]:
C = 0.46
D = 0.34
E = 0.08
F = 0.10

scaling_factor = (
    C * (100 - low_clouds) / 100
    + D * (100 - mid_clouds) / 100
    + E * (100 - high_clouds) / 100
    + F
)

### Calculate solar incident angle


In [ ]:
class cos_solar_incident_angle:
    def __init__(self):
        self.tilt_angle = math.radians(5)
        self.azimuth_angle = math.radians(0)
        self.set_day_of_year()
        self.set_equation_of_time()
        self.set_time_correction_factor()
        self.set_ast_corrected_local_solar_time()
        self.set_solar_declination()
        self.set_hour_angle()
        self.set_cos_incident_angle()

    def set_day_of_year(self):
        self.day_of_year = datetime.now().timetuple().tm_yday

    def set_equation_of_time(self):
        B = 360 * (self.day_of_year - 81) / 365
        self.equation_of_time = (
            9.87 * math.sin(math.radians(2 * B))
            - 7.53 * math.cos(math.radians(B))
            - 1.5 * math.sin(math.radians(B))
        )  # minutes

    def set_time_correction_factor(self):  # 8 refers to UTC+8
        self.time_correction_factor = (
            4 * (LONG - 8 * 15) + self.equation_of_time
        )

    def set_ast_corrected_local_solar_time(self):
        self.ast_corrected_local_solar_time = (
            datetime.now().hour + datetime.now().minute / 60
        ) + self.time_correction_factor / 60  # hours

    def set_solar_declination(self):
        self.solar_declination = 23.45 * math.sin(
            math.radians(360 * (self.day_of_year + 284) / 365)
        )
        self.solar_declination = math.radians(self.solar_declination)

    def set_hour_angle(self):
        self.hour_angle = 15 * (self.ast_corrected_local_solar_time - 12)
        self.hour_angle = math.radians(self.hour_angle)

    def set_cos_incident_angle(self):
        delta = self.solar_declination
        beta = self.tilt_angle
        zeta = self.azimuth_angle
        h = self.hour_angle
        self.cos_incident_angle = (
            (math.sin(LONG) * math.sin(delta))
            - (math.cos(beta))
            - (math.cos(LONG) * math.sin(delta) * math.sin(beta) * math.cos(zeta))
            + (math.cos(LONG) * math.cos(delta) * math.cos(h) * math.cos(beta))
            + (math.sin(LONG) * math.cos(delta) * math.cos(h) * math.sin(beta) * math.cos(zeta))
            + (math.cos(delta) * math.sin(h) * math.sin(beta) * math.sin(zeta))
        )
